# Apply FT

In [2]:
#%cd ./flosp
import pandas as pd
import numpy as np
import seaborn as sns
import featuretools as ft
#import flosp
#import bradlib as bl
sns.set()

from core import create_timeseries_from_events, make_callender_columns

# get data

In [3]:
ED = pd.read_pickle('./../3_Data/SGH_ED_bk.pkl')

In [4]:
IP = pd.read_pickle('./../3_Data/SGH_ward_BK.pkl')

In [5]:
df = make_callender_columns(ED,'arrival','arr')

In [6]:
i = 2000

In [7]:
ED.head(2)

,pseudo_patient_number,attendance_number,age,gender,department,site_attended,arrival,arrival_mode,arrival_mode_desc,1st_time_triaged_[datetime],...,age_group,arr_hour,arr_dayofweek,arr_day,arr_month,arr_week,arr_dayofweek_name,arr_year,arr_date,arr_flag_wkend
0,4101045105,90902325901,94.0,F,Main ED,Main ED,2009-04-13 09:16:00,NaN,BROUGHT IN BY AMBULANCE,NaT,...,75+,9,0,13,4,16,Monday,2009,2009-04-13,0
1,644868441371,90902326101,47.0,M,Main ED,Main ED,2009-04-13 09:22:00,NaN,BROUGHT IN BY AMBULANCE,NaT,...,36-64,9,0,13,4,16,Monday,2009,2009-04-13,0


In [8]:
ED.isnull().sum()

pseudo_patient_number                 1
attendance_number                     0
age                                  10
gender                               10
department                            0
site_attended                         0
arrival                               0
arrival_mode                     269946
arrival_mode_desc                     0
1st_time_triaged_[datetime]      280630
1st_time_seen_by_doctor           76978
time_of_inpatient_bed_request    658458
time_of_speciality_referral      653702
speciality_referred_to           653703
time_into_cdu                    816058
time_leaving_dept_inc_cdu             0
admission_flag                        0
majorsminorsresus_patient        114766
triage_category                  639560
age_group                            10
arr_hour                              0
arr_dayofweek                         0
arr_day                               0
arr_month                             0
arr_week                              0


# clean ED

In [9]:
ED.arrival_mode_desc.value_counts()

OTHER                      566738
BROUGHT IN BY AMBULANCE    347099
Name: arrival_mode_desc, dtype: int64

In [10]:
ED['arrival_mode_desc_bool'] = ED.arrival_mode_desc == 'BROUGHT IN BY AMBULANCE'

In [11]:
ED['arrival_mode_desc_bool'].value_counts()

False    566738
True     347099
Name: arrival_mode_desc_bool, dtype: int64

In [12]:
ED['admission_flag_bool'] = ED.admission_flag == 1

In [13]:
ED['admission_flag_bool'].value_counts()

False    559945
True     353892
Name: admission_flag_bool, dtype: int64

In [14]:
ED.columns

Index(['pseudo_patient_number', 'attendance_number', 'age', 'gender',
       'department', 'site_attended', 'arrival', 'arrival_mode',
       'arrival_mode_desc', '1st_time_triaged_[datetime]',
       '1st_time_seen_by_doctor', 'time_of_inpatient_bed_request',
       'time_of_speciality_referral', 'speciality_referred_to',
       'time_into_cdu', 'time_leaving_dept_inc_cdu', 'admission_flag',
       'majorsminorsresus_patient', 'triage_category', 'age_group', 'arr_hour',
       'arr_dayofweek', 'arr_day', 'arr_month', 'arr_week',
       'arr_dayofweek_name', 'arr_year', 'arr_date', 'arr_flag_wkend',
       'arrival_mode_desc_bool', 'admission_flag_bool'],
      dtype='object')

In [15]:
ED['wait_time_total'] = (ED['time_leaving_dept_inc_cdu'] - ED['arrival'])/pd.Timedelta(1,'m')

# create es

In [16]:
es = ft.EntitySet(id='activity')

In [17]:
es

Entityset: activity
  Entities:
  Relationships:
    No relationships

In [18]:
ED.arrival.dtypes

dtype('<M8[ns]')

In [19]:
ED.columns

Index(['pseudo_patient_number', 'attendance_number', 'age', 'gender',
       'department', 'site_attended', 'arrival', 'arrival_mode',
       'arrival_mode_desc', '1st_time_triaged_[datetime]',
       '1st_time_seen_by_doctor', 'time_of_inpatient_bed_request',
       'time_of_speciality_referral', 'speciality_referred_to',
       'time_into_cdu', 'time_leaving_dept_inc_cdu', 'admission_flag',
       'majorsminorsresus_patient', 'triage_category', 'age_group', 'arr_hour',
       'arr_dayofweek', 'arr_day', 'arr_month', 'arr_week',
       'arr_dayofweek_name', 'arr_year', 'arr_date', 'arr_flag_wkend',
       'arrival_mode_desc_bool', 'admission_flag_bool', 'wait_time_total'],
      dtype='object')

In [20]:
df_ed = ED[['arrival','arr_date','attendance_number','pseudo_patient_number',
            'age','gender','arrival_mode_desc_bool',
                    'time_leaving_dept_inc_cdu',
            'admission_flag_bool','wait_time_total'
                   ]]

for i in ED.columns:
    print(i)
    print(ED[i].dtype)

In [21]:
df_ed.head()

,arrival,arr_date,attendance_number,pseudo_patient_number,age,gender,arrival_mode_desc_bool,time_leaving_dept_inc_cdu,admission_flag_bool,wait_time_total
0,2009-04-13 09:16:00,2009-04-13,90902325901,4101045105,94.0,F,True,2009-04-13 12:24:00,False,188.0
1,2009-04-13 09:22:00,2009-04-13,90902326101,644868441371,47.0,M,True,2009-04-13 12:55:00,True,213.0
2,2009-04-13 09:31:00,2009-04-13,90902326501,668666687357,0.0,M,False,2009-04-13 11:54:00,True,143.0
3,2009-04-13 10:52:00,2009-04-13,90902328601,446747461391,4.0,M,False,2009-04-13 13:49:00,False,177.0
4,2009-04-13 11:18:00,2009-04-13,90902328801,423646238168,31.0,F,True,2009-04-13 14:49:00,True,211.0


In [22]:
df_ed.tail()

,arrival,arr_date,attendance_number,pseudo_patient_number,age,gender,arrival_mode_desc_bool,time_leaving_dept_inc_cdu,admission_flag_bool,wait_time_total
1124659,2018-05-04 12:00:00,2018-05-04,91803566401,497545978018,69.0,F,True,2018-05-04 15:04:00,False,184.0
1124671,2018-05-26 11:31:00,2018-05-26,91804244501,812787124394,73.0,M,True,2018-05-26 12:40:00,True,69.0
1124673,2018-05-04 19:37:00,2018-05-04,91803579801,073505730740,2.0,F,False,2018-05-04 23:09:00,False,212.0
1124681,2018-05-27 06:17:00,2018-05-27,91804272201,757373573113,93.0,M,True,2018-05-27 18:11:00,True,714.0
1124686,2018-05-27 00:13:00,2018-05-27,91804267301,854787547747,37.0,M,True,2018-05-27 10:04:00,False,591.0


In [23]:
ED['time_leaving_dept_inc_cdu'].max()

Timestamp('2018-10-01 18:53:00')

In [24]:
df_ed.loc[0].arrival

Timestamp('2009-04-13 09:16:00')

In [25]:
df_ed.arrival.dtype

dtype('<M8[ns]')

In [26]:
ft.variable_types.Numeric

featuretools.variable_types.variable.Numeric

In [27]:
es = es.entity_from_dataframe(entity_id='EDatt',
                        dataframe=df_ed[0:2000],
                    index='attendance_number',
                        time_index='arrival',
                              secondary_time_index={'time_leaving_dept_inc_cdu':['admission_flag_bool','wait_time_total']},
                             variable_types={'arrival':ft.variable_types.Datetime,
                                            'arr_date':ft.variable_types.Datetime,
                                             'pseudo_patient_number':ft.variable_types.Numeric,
                                             'age':ft.variable_types.Numeric,
                                             'attendance_number':ft.variable_types.Numeric
})

#### make patient table

In [28]:
es = es.normalize_entity(base_entity_id='EDatt',
                   new_entity_id ='EDpatients',
                   index='pseudo_patient_number',
                         make_time_index=True,
                   additional_variables=['gender'])

In [29]:
es['EDpatients'].df.head()

,pseudo_patient_number,gender,first_EDatt_time
211525117317,211525117317,F,2009-04-01 02:18:00
314131140110,314131140110,F,2009-04-01 06:11:00
655565556386,655565556386,F,2009-04-01 08:03:00
607161071001,607161071001,F,2009-04-01 08:20:00
257929577127,257929577127,M,2009-04-01 09:27:00


In [30]:
es['EDatt'].df['arrival'].head()

90902042301   2009-04-01 02:18:00
90902043501   2009-04-01 06:11:00
90902044101   2009-04-01 08:03:00
90902044201   2009-04-01 08:20:00
90902045601   2009-04-01 09:27:00
Name: arrival, dtype: datetime64[ns]

In [31]:
es['EDatt'].last_time_index = es['EDatt'].df['time_leaving_dept_inc_cdu']

In [32]:
es['EDatt'].last_time_index.head()

90902042301   2009-04-01 03:11:00
90902043501   2009-04-01 10:08:00
90902044101   2009-04-01 10:22:00
90902044201   2009-04-01 08:46:00
90902045601   2009-04-01 12:49:00
Name: time_leaving_dept_inc_cdu, dtype: datetime64[ns]

In [33]:
es['EDatt'].secondary_time_index

{'time_leaving_dept_inc_cdu': ['admission_flag_bool',
  'wait_time_total',
  'time_leaving_dept_inc_cdu']}

In [34]:
es['EDatt'].df.head(2)

,attendance_number,arrival_mode_desc_bool,time_leaving_dept_inc_cdu,admission_flag_bool,wait_time_total,arrival,arr_date,pseudo_patient_number,age
90902042301,90902042301,True,2009-04-01 03:11:00,False,53.0,2009-04-01 02:18:00,2009-04-01,211525117317,30.0
90902043501,90902043501,True,2009-04-01 10:08:00,True,237.0,2009-04-01 06:11:00,2009-04-01,314131140110,87.0


#### must create last time for the use of training windows

datetime.datetime(2018, 12, 5, 15, 39, 35, 761878)

In [222]:
es['EDpatients'].df['last_time'] = pd.datetime.now()

In [223]:
es['EDpatients'].last_time_index = es['EDpatients'].df['last_time']

### create hourly - atten_no realationship

In [224]:
hrly_atten_rel = es['EDatt'].df[['arrival','time_leaving_dept_inc_cdu']]
hrly_atten_rel['arrival'] = hrly_atten_rel['arrival'].apply(lambda x : x.replace(second=0,minute=0)) # round arrival hour down
hrly_atten_rel['time_leaving_dept_inc_cdu'] = hrly_atten_rel['time_leaving_dept_inc_cdu'].apply(lambda x : x.replace(second=0,minute=0)) +pd.Timedelta(hours=1) # round leaving tim up
hrly_atten_rel.head()

,arrival,time_leaving_dept_inc_cdu
90902042301,2009-04-01 02:00:00,2009-04-01 04:00:00
90902043501,2009-04-01 06:00:00,2009-04-01 11:00:00
90902044101,2009-04-01 08:00:00,2009-04-01 11:00:00
90902044201,2009-04-01 08:00:00,2009-04-01 09:00:00
90902045601,2009-04-01 09:00:00,2009-04-01 13:00:00


In [225]:
hrly_atten_rel.head()

,arrival,time_leaving_dept_inc_cdu
90902042301,2009-04-01 02:00:00,2009-04-01 04:00:00
90902043501,2009-04-01 06:00:00,2009-04-01 11:00:00
90902044101,2009-04-01 08:00:00,2009-04-01 11:00:00
90902044201,2009-04-01 08:00:00,2009-04-01 09:00:00
90902045601,2009-04-01 09:00:00,2009-04-01 13:00:00


In [226]:
hrly_atten_rel['n_hours'] = ((hrly_atten_rel['time_leaving_dept_inc_cdu'] - hrly_atten_rel['arrival'])/pd.Timedelta(1,'h')).astype(int)

In [227]:
hrly_atten_rel.head()

,arrival,time_leaving_dept_inc_cdu,n_hours
90902042301,2009-04-01 02:00:00,2009-04-01 04:00:00,2
90902043501,2009-04-01 06:00:00,2009-04-01 11:00:00,5
90902044101,2009-04-01 08:00:00,2009-04-01 11:00:00,3
90902044201,2009-04-01 08:00:00,2009-04-01 09:00:00,1
90902045601,2009-04-01 09:00:00,2009-04-01 13:00:00,4


In [228]:
import itertools

In [229]:
#### create a (long format) list of links between attendance numbers and 
#%%timeit



# function for list comp which finds list of datetimes (for each hour)
date_func = lambda datetime , offset : datetime + pd.Timedelta(offset,'h')

# iterate over rows in df
sample = hrly_atten_rel.head()
sample = sample.reset_index()

ids = np.empty(shape=(sample['n_hours'].sum()),dtype='int64') # initilise array - change to np.empty() to speed up
timestamps = np.empty(shape=(sample['n_hours'].sum()),dtype='datetime64[s]')
row_count = 0

for row in sample.itertuples():
    atten_id = [row[1]]
    hour_list = [date_func(row[2],i) for i in np.arange(row[4])] # creates list of hour datetimes
    
    # create array of list for all combinations of timestamp
    for i in itertools.product(atten_id,hour_list):
        ids[row_count] = i[0] # assign patient numbers
        timestamps[row_count] = i[1]
        row_count += 1 # add to row count for new array
        
print(ids)
print(timestamps)
ids.shape

[90902042301 90902042301 90902043501 90902043501 90902043501 90902043501
 90902043501 90902044101 90902044101 90902044101 90902044201 90902045601
 90902045601 90902045601 90902045601]
['2009-04-01T02:00:00' '2009-04-01T03:00:00' '2009-04-01T06:00:00'
 '2009-04-01T07:00:00' '2009-04-01T08:00:00' '2009-04-01T09:00:00'
 '2009-04-01T10:00:00' '2009-04-01T08:00:00' '2009-04-01T09:00:00'
 '2009-04-01T10:00:00' '2009-04-01T08:00:00' '2009-04-01T09:00:00'
 '2009-04-01T10:00:00' '2009-04-01T11:00:00' '2009-04-01T12:00:00']


(15,)

In [230]:
data = {'ids':ids,
       'timestamp':timestamps}

pd.DataFrame(data=data)

,ids,timestamp
0,90902042301,2009-04-01 02:00:00
1,90902042301,2009-04-01 03:00:00
2,90902043501,2009-04-01 06:00:00
3,90902043501,2009-04-01 07:00:00
4,90902043501,2009-04-01 08:00:00
5,90902043501,2009-04-01 09:00:00
6,90902043501,2009-04-01 10:00:00
7,90902044101,2009-04-01 08:00:00
8,90902044101,2009-04-01 09:00:00
9,90902044101,2009-04-01 10:00:00


In [231]:
fm, features = ft.dfs(entityset=es,
                       target_entity='EDatt',
                       cutoff_time=ct,
                       cutoff_time_in_index=True,
                     max_depth=1,
                     training_window="1 hours")

fm

arrival_mode_desc_bool  \
attendance_number time                                          
90902042301       2009-04-01 02:00:00                     NaN   
                  2009-04-01 03:00:00                     1.0   
                  2009-04-01 04:00:00                     1.0   
                  2009-04-01 05:00:00                     NaN   
90902043501       2009-04-01 07:00:00                     1.0   
                  2014-01-01 04:00:00                     NaN   

                                       pseudo_patient_number   age  \
attendance_number time                                               
90902042301       2009-04-01 02:00:00                    NaN   NaN   
                  2009-04-01 03:00:00           2.115251e+11  30.0   
                  2009-04-01 04:00:00           2.115251e+11  30.0   
                  2009-04-01 05:00:00                    NaN   NaN   
90902043501       2009-04-01 07:00:00           3.141311e+11  87.0   
                  2014-01-01 04:00:00                    NaN   NaN   

                                       DAY(time_leaving_dept_inc_cdu)  \
attendance_number time                                                  
90902042301       2009-04-01 02:00:00                             NaN   
                  2009-04-01 03:00:00                             NaN   
                  2009-04-01 04:00:00                             1.0   
                  2009-04-01 05:00:00                             NaN   
90902043501       2009-04-01 07:00:00                             NaN   
                  2014-01-01 04:00:00                             NaN   

                                       DAY(arrival)  DAY(arr_date)  \
attendance_number time                                               
90902042301       2009-04-01 02:00:00           NaN            NaN   
                  2009-04-01 03:00:00           1.0            1.0   
                  2009-04-01 04:00:00           1.0            1.0   
                  2009-04-01 05:00:00           NaN            NaN   
90902043501       2009-04-01 07:00:00           1.0            1.0   
                  2014-01-01 04:00:00           NaN            NaN   

                                       YEAR(time_leaving_dept_inc_cdu)  \
attendance_number time                                                   
90902042301       2009-04-01 02:00:00                              NaN   
                  2009-04-01 03:00:00                              NaN   
                  2009-04-01 04:00:00                           2009.0   
                  2009-04-01 05:00:00                              NaN   
90902043501       2009-04-01 07:00:00                              NaN   
                  2014-01-01 04:00:00                              NaN   

                                       YEAR(arrival)  YEAR(arr_date)  \
attendance_number time                                                 
90902042301       2009-04-01 02:00:00            NaN             NaN   
                  2009-04-01 03:00:00         2009.0          2009.0   
                  2009-04-01 04:00:00         2009.0          2009.0   
                  2009-04-01 05:00:00            NaN             NaN   
90902043501       2009-04-01 07:00:00         2009.0          2009.0   
                  2014-01-01 04:00:00            NaN             NaN   

                                       MONTH(time_leaving_dept_inc_cdu)  \
attendance_number time                                                    
90902042301       2009-04-01 02:00:00                               NaN   
                  2009-04-01 03:00:00                               NaN   
                  2009-04-01 04:00:00                               4.0   
                  2009-04-01 05:00:00                               NaN   
90902043501       2009-04-01 07:00:00                               NaN   
                  2014-01-01 04:00:00                               NaN   

                                       M

In [176]:
#### put back into df
pd.DataFrame(data=[ids,timestamps],columns=['atten_id','timestamp'])

ValueError: Must pass 2-d input

In [141]:
sample.reset_index(inplace=True)

,index,arrival,time_leaving_dept_inc_cdu,n_hours
0,90902042301,2009-04-01 02:00:00,2009-04-01 04:00:00,2
1,90902043501,2009-04-01 06:00:00,2009-04-01 11:00:00,5
2,90902044101,2009-04-01 08:00:00,2009-04-01 11:00:00,3
3,90902044201,2009-04-01 08:00:00,2009-04-01 09:00:00,1
4,90902045601,2009-04-01 09:00:00,2009-04-01 13:00:00,4


In [149]:

#X = np.array()
for i in itertools.product(['1'],['2','3','4']):
    print(list(i))
    X[row[0]] = i
    
X

['1', '2']
['1', '3']
['1', '4']


array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [1., 4.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [110]:

X.append()

In [109]:
X = np.array([[1,2,3]])
X = np.append(X,[[3,4,5]],axis=0)
X

array([[1, 2, 3],
       [3, 4, 5]])

In [35]:
ct=pd.DataFrame()

ct['attendance_number'] = [90902042301,90902042301,90902042301,90902042301,90902043501,90902043501]

ct['time'] = pd.to_datetime(['2009-4-1 02:00',
                             '2009-4-1 03:00',
                             '2009-4-1 04:00',
                             '2009-4-1 05:00',
                              '2009-4-1 07:00',
                              '2014-1-1 04:00'])

In [36]:
ct

,attendance_number,time
0,90902042301,2009-04-01 02:00:00
1,90902042301,2009-04-01 03:00:00
2,90902042301,2009-04-01 04:00:00
3,90902042301,2009-04-01 05:00:00
4,90902043501,2009-04-01 07:00:00
5,90902043501,2014-01-01 04:00:00


In [70]:
es['EDpatients'].last_time_index = es['EDpatients'].df['last_time']

In [71]:
fm, features = ft.dfs(entityset=es,
                       target_entity='EDatt',
                       cutoff_time=ct,
                       cutoff_time_in_index=True,
                     max_depth=1,
                     training_window="1 hours")

fm

arrival_mode_desc_bool  \
attendance_number time                                          
90902042301       2009-04-01 02:00:00                     NaN   
                  2009-04-01 03:00:00                     1.0   
                  2009-04-01 04:00:00                     1.0   
                  2009-04-01 05:00:00                     NaN   
90902043501       2009-04-01 07:00:00                     1.0   
                  2014-01-01 04:00:00                     NaN   

                                       pseudo_patient_number   age  \
attendance_number time                                               
90902042301       2009-04-01 02:00:00                    NaN   NaN   
                  2009-04-01 03:00:00           2.115251e+11  30.0   
                  2009-04-01 04:00:00           2.115251e+11  30.0   
                  2009-04-01 05:00:00                    NaN   NaN   
90902043501       2009-04-01 07:00:00           3.141311e+11  87.0   
                  2014-01-01 04:00:00                    NaN   NaN   

                                       DAY(time_leaving_dept_inc_cdu)  \
attendance_number time                                                  
90902042301       2009-04-01 02:00:00                             NaN   
                  2009-04-01 03:00:00                             NaN   
                  2009-04-01 04:00:00                             1.0   
                  2009-04-01 05:00:00                             NaN   
90902043501       2009-04-01 07:00:00                             NaN   
                  2014-01-01 04:00:00                             NaN   

                                       DAY(arrival)  DAY(arr_date)  \
attendance_number time                                               
90902042301       2009-04-01 02:00:00           NaN            NaN   
                  2009-04-01 03:00:00           1.0            1.0   
                  2009-04-01 04:00:00           1.0            1.0   
                  2009-04-01 05:00:00           NaN            NaN   
90902043501       2009-04-01 07:00:00           1.0            1.0   
                  2014-01-01 04:00:00           NaN            NaN   

                                       YEAR(time_leaving_dept_inc_cdu)  \
attendance_number time                                                   
90902042301       2009-04-01 02:00:00                              NaN   
                  2009-04-01 03:00:00                              NaN   
                  2009-04-01 04:00:00                           2009.0   
                  2009-04-01 05:00:00                              NaN   
90902043501       2009-04-01 07:00:00                              NaN   
                  2014-01-01 04:00:00                              NaN   

                                       YEAR(arrival)  YEAR(arr_date)  \
attendance_number time                                                 
90902042301       2009-04-01 02:00:00            NaN             NaN   
                  2009-04-01 03:00:00         2009.0          2009.0   
                  2009-04-01 04:00:00         2009.0          2009.0   
                  2009-04-01 05:00:00            NaN             NaN   
90902043501       2009-04-01 07:00:00         2009.0          2009.0   
                  2014-01-01 04:00:00            NaN             NaN   

                                       MONTH(time_leaving_dept_inc_cdu)  \
attendance_number time                                                    
90902042301       2009-04-01 02:00:00                               NaN   
                  2009-04-01 03:00:00                               NaN   
                  2009-04-01 04:00:00                               4.0   
                  2009-04-01 05:00:00                               NaN   
90902043501       2009-04-01 07:00:00                               NaN   
                  2014-01-01 04:00:00                               NaN   

                                       M

In [62]:
es

Entityset: activity
  Entities:
    EDatt [Rows: 2000, Columns: 9]
    EDpatients [Rows: 1962, Columns: 4]
  Relationships:
    EDatt.pseudo_patient_number -> EDpatients.pseudo_patient_number

In [63]:
ct2 = es['EDatt'].df[['attendance_number','arrival','arrival_mode_desc_bool']].head(5)

In [65]:
fm, features = ft.dfs(entityset=es,
                       target_entity='EDatt',
                       cutoff_time=ct2,
                       cutoff_time_in_index=True,
                     max_depth=2)

fm

,,arrival_mode_desc_bool,pseudo_patient_number,age,DAY(time_leaving_dept_inc_cdu),DAY(arrival),DAY(arr_date),YEAR(time_leaving_dept_inc_cdu),YEAR(arrival),YEAR(arr_date),MONTH(time_leaving_dept_inc_cdu),...,EDpatients.MIN(EDatt.age),EDpatients.MEAN(EDatt.wait_time_total),EDpatients.MEAN(EDatt.age),EDpatients.COUNT(EDatt),EDpatients.PERCENT_TRUE(EDatt.arrival_mode_desc_bool),EDpatients.PERCENT_TRUE(EDatt.admission_flag_bool),EDpatients.DAY(first_EDatt_time),EDpatients.YEAR(first_EDatt_time),EDpatients.MONTH(first_EDatt_time),EDpatients.WEEKDAY(first_EDatt_time)
attendance_number,time,,,,,,,,,,,,,,,,,,,,,
90902042301,2009-04-01 02:18:00,True,211525117317,30.0,NaN,1,1,NaN,2009,2009,NaN,...,30.0,NaN,30.0,1,1.0,0.0,1,2009,4,2
90902043501,2009-04-01 06:11:00,True,314131140110,87.0,NaN,1,1,NaN,2009,2009,NaN,...,87.0,NaN,87.0,1,1.0,0.0,1,2009,4,2
90902044101,2009-04-01 08:03:00,False,655565556386,5.0,NaN,1,1,NaN,2009,2009,NaN,...,5.0,NaN,5.0,1,0.0,0.0,1,2009,4,2
90902044201,2009-04-01 08:20:00,False,607161071001,55.0,NaN,1,1,NaN,2009,2009,NaN,...,55.0,NaN,55.0,1,0.0,0.0,1,2009,4,2
90902045601,2009-04-01 09:27:00,False,257929577127,15.0,NaN,1,1,NaN,2009,2009,NaN,...,15.0,NaN,15.0,1,0.0,0.0,1,2009,4,2


In [32]:
break

SyntaxError: 'break' outside loop (<ipython-input-32-6aaf1f276005>, line 1)

#### create a day table - by normalising a new feature

In [28]:
es = es.normalize_entity(base_entity_id='EDatt',
                   new_entity_id="days",
                    index="arr_date")

#### import hourly table

In [28]:
hrly = pd.read_pickle('./../3_Data/processed/sgh/sghHOURLY.pkl')

In [29]:
hrly.columns

Index(['EDocc_nonadmit', 'EDocc_admit', 'EDocc_total', 'EDocc_awaitingadm',
       'EDocc_nonbreach', 'EDocc_breach', 'IPocc_NonElective',
       'IPocc_Day_Case', 'IPocc_Elective', 'IPocc_total', 'IPocc_elec_nonelec',
       'IPadm', 'IPadm_nonelec', 'IPadm_daycase', 'IPadm_elective', 'IPdis',
       'IPdis_nonelec', 'IPdis_daycase', 'IPdis_elective',
       'IPadm_elec_nonelec', 'IPdis_elec_nonelec', 'EDarrive',
       'EDarrive_breach', 'EDarrive_adm', 'EDdepart', 'EDdepart_breach',
       'EDdepart_adm', 'datetime', 'dt_hour', 'dt_dayofweek', 'dt_day',
       'dt_month', 'dt_week', 'dt_dayofweek_name', 'dt_year', 'dt_date',
       'dt_flag_wkend'],
      dtype='object')

In [30]:
hrly2 = hrly[['dt_date','EDocc_nonadmit', 'EDocc_admit', 'EDocc_total', 'EDocc_awaitingadm',
       'EDocc_nonbreach', 'EDocc_breach', 'IPocc_NonElective',
       'IPocc_Day_Case', 'IPocc_Elective', 'IPocc_total', 'IPocc_elec_nonelec',
       'IPadm', 'IPadm_nonelec', 'IPadm_daycase', 'IPadm_elective', 'IPdis',
       'IPdis_nonelec', 'IPdis_daycase', 'IPdis_elective',
       'IPadm_elec_nonelec', 'IPdis_elec_nonelec', 'EDarrive',
       'EDarrive_breach', 'EDarrive_adm', 'EDdepart', 'EDdepart_breach',
       'EDdepart_adm']].reset_index()

In [31]:
es.entity_from_dataframe(entity_id='hourly',dataframe=hrly2,index='index')

Entityset: activity
  Entities:
    EDatt [Rows: 913837, Columns: 7]
    patients [Rows: 399661, Columns: 3]
    days [Rows: 3470, Columns: 2]
    hourly [Rows: 83295, Columns: 29]
  Relationships:
    EDatt.pseudo_patient_number -> patients.pseudo_patient_number
    EDatt.arr_date -> days.arr_date

In [32]:
new_relationship = ft.Relationship(es["days"]["arr_date"],
                                    es["hourly"]["dt_date"])

es = es.add_relationship(new_relationship)

In [33]:
es['hourly'].df.head(2)

,index,dt_date,EDocc_nonadmit,EDocc_admit,EDocc_total,EDocc_awaitingadm,EDocc_nonbreach,EDocc_breach,IPocc_NonElective,IPocc_Day_Case,...,IPdis_daycase,IPdis_elective,IPadm_elec_nonelec,IPdis_elec_nonelec,EDarrive,EDarrive_breach,EDarrive_adm,EDdepart,EDdepart_breach,EDdepart_adm
2009-04-01 00:00:00,2009-04-01 00:00:00,2009-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,0.0,0.0,0.0
2009-04-01 01:00:00,2009-04-01 01:00:00,2009-04-01,2.0,4.0,6.0,0.0,3.0,3.0,3.0,2.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,1.0,0.0,0.0


# summary of esets

In [34]:
es

Entityset: activity
  Entities:
    EDatt [Rows: 913837, Columns: 7]
    patients [Rows: 399661, Columns: 3]
    days [Rows: 3470, Columns: 2]
    hourly [Rows: 83295, Columns: 29]
  Relationships:
    EDatt.pseudo_patient_number -> patients.pseudo_patient_number
    EDatt.arr_date -> days.arr_date
    hourly.dt_date -> days.arr_date

In [35]:
es['days'].df.head()

,arr_date,first_EDatt_time
2009-04-01,2009-04-01,2009-04-01 00:04:00
2009-04-02,2009-04-02,2009-04-02 00:05:00
2009-04-03,2009-04-03,2009-04-03 00:01:00
2009-04-04,2009-04-04,2009-04-04 00:04:00
2009-04-05,2009-04-05,2009-04-05 00:03:00


In [36]:
es['EDatt'].df.head(2)

,attendance_number,arrival_mode_desc_bool,admission_flag,arrival,arr_date,pseudo_patient_number,age
90902041001,90902041001,False,0,2009-04-01 00:04:00,2009-04-01,4.930409e+11,19.0
90902041101,90902041101,True,0,2009-04-01 00:19:00,2009-04-01,6.104641e+11,59.0


In [37]:
es['patients'].df.head(3)

,pseudo_patient_number,gender,first_EDatt_time
4.930409e+11,4.930409e+11,F,2009-04-01 00:04:00
6.104641e+11,6.104641e+11,F,2009-04-01 00:19:00
7.723737e+11,7.723737e+11,M,2009-04-01 00:19:00


In [38]:
es['hourly'].df.head(2)

,index,dt_date,EDocc_nonadmit,EDocc_admit,EDocc_total,EDocc_awaitingadm,EDocc_nonbreach,EDocc_breach,IPocc_NonElective,IPocc_Day_Case,...,IPdis_daycase,IPdis_elective,IPadm_elec_nonelec,IPdis_elec_nonelec,EDarrive,EDarrive_breach,EDarrive_adm,EDdepart,EDdepart_breach,EDdepart_adm
2009-04-01 00:00:00,2009-04-01 00:00:00,2009-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,0.0,0.0,0.0
2009-04-01 01:00:00,2009-04-01 01:00:00,2009-04-01,2.0,4.0,6.0,0.0,3.0,3.0,3.0,2.0,...,0.0,0.0,0.0,0.0,6.0,3.0,4.0,1.0,0.0,0.0


In [39]:
es['days'].df.head(3)

,arr_date,first_EDatt_time
2009-04-01,2009-04-01,2009-04-01 00:04:00
2009-04-02,2009-04-02,2009-04-02 00:05:00
2009-04-03,2009-04-03,2009-04-03 00:01:00


# make features

feature_matrix, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum'],
      trans_primitives=['month','year','day'],
    max_depth=1,n_jobs=2)

feature_matrix

In [63]:
feature_matrix1, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum','max','last'],
      trans_primitives=['month','year','weekend','weekday'],
    max_depth=1,n_jobs=1)

feature_matrix1.shape

(3470, 125)

In [64]:
feature_matrix2, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum','max','last'],
      trans_primitives=['month','year','weekend','weekday'],
    max_depth=2,n_jobs=1)

feature_matrix2.shape

(3470, 135)

In [65]:
feature_matrix, feature_defs = ft.dfs(entityset=es,target_entity='days',
      agg_primitives=['count','mean','num_true','sum','max','last'],
      trans_primitives=['month','year','weekend','weekday'],
    max_depth=3,n_jobs=1)

feature_matrix.shape

(3470, 179)

In [66]:
set(feature_matrix.columns) - set(feature_matrix1.columns)

{'LAST(EDatt.MONTH(arrival))',
 'LAST(EDatt.WEEKDAY(arrival))',
 'LAST(EDatt.WEEKEND(arrival))',
 'LAST(EDatt.YEAR(arrival))',
 'LAST(EDatt.patients.COUNT(EDatt))',
 'LAST(EDatt.patients.LAST(EDatt.admission_flag))',
 'LAST(EDatt.patients.LAST(EDatt.age))',
 'LAST(EDatt.patients.LAST(EDatt.arr_date))',
 'LAST(EDatt.patients.MAX(EDatt.admission_flag))',
 'LAST(EDatt.patients.MAX(EDatt.age))',
 'LAST(EDatt.patients.MEAN(EDatt.admission_flag))',
 'LAST(EDatt.patients.MEAN(EDatt.age))',
 'LAST(EDatt.patients.MONTH(first_EDatt_time))',
 'LAST(EDatt.patients.NUM_TRUE(EDatt.arrival_mode_desc_bool))',
 'LAST(EDatt.patients.SUM(EDatt.admission_flag))',
 'LAST(EDatt.patients.SUM(EDatt.age))',
 'LAST(EDatt.patients.WEEKDAY(first_EDatt_time))',
 'LAST(EDatt.patients.YEAR(first_EDatt_time))',
 'LAST(EDatt.patients.gender)',
 'MAX(EDatt.patients.COUNT(EDatt))',
 'MAX(EDatt.patients.LAST(EDatt.admission_flag))',
 'MAX(EDatt.patients.LAST(EDatt.age))',
 'MAX(EDatt.patients.MAX(EDatt.admission_flag))',

In [41]:
for i in feature_matrix.columns:
    print(i)

COUNT(EDatt)
MEAN(EDatt.admission_flag)
MEAN(EDatt.age)
NUM_TRUE(EDatt.arrival_mode_desc_bool)
SUM(EDatt.admission_flag)
SUM(EDatt.age)
MAX(EDatt.admission_flag)
MAX(EDatt.age)
LAST(EDatt.arrival_mode_desc_bool)
LAST(EDatt.admission_flag)
LAST(EDatt.pseudo_patient_number)
LAST(EDatt.age)
COUNT(hourly)
MEAN(hourly.EDocc_nonadmit)
MEAN(hourly.EDocc_admit)
MEAN(hourly.EDocc_total)
MEAN(hourly.EDocc_awaitingadm)
MEAN(hourly.EDocc_nonbreach)
MEAN(hourly.EDocc_breach)
MEAN(hourly.IPocc_NonElective)
MEAN(hourly.IPocc_Day_Case)
MEAN(hourly.IPocc_Elective)
MEAN(hourly.IPocc_total)
MEAN(hourly.IPocc_elec_nonelec)
MEAN(hourly.IPadm)
MEAN(hourly.IPadm_nonelec)
MEAN(hourly.IPadm_daycase)
MEAN(hourly.IPadm_elective)
MEAN(hourly.IPdis)
MEAN(hourly.IPdis_nonelec)
MEAN(hourly.IPdis_daycase)
MEAN(hourly.IPdis_elective)
MEAN(hourly.IPadm_elec_nonelec)
MEAN(hourly.IPdis_elec_nonelec)
MEAN(hourly.EDarrive)
MEAN(hourly.EDarrive_breach)
MEAN(hourly.EDarrive_adm)
MEAN(hourly.EDdepart)
MEAN(hourly.EDdepart_bre

In [42]:
feature_matrix.shape

(3470, 179)

In [43]:
feature_matrix.shape

(3470, 179)

## do some basic drops

In [44]:
uniques = feature_matrix.apply(lambda x: x.nunique())

In [45]:
uniques[uniques==1].index

Index(['MAX(EDatt.admission_flag)', 'COUNT(hourly)',
       'MAX(EDatt.patients.MEAN(EDatt.admission_flag))',
       'MAX(EDatt.patients.MAX(EDatt.admission_flag))',
       'MAX(EDatt.patients.LAST(EDatt.admission_flag))'],
      dtype='object')

In [46]:
feature_matrix = feature_matrix.drop(uniques[uniques==1].index, axis=1)

#### drop columns which have all same values

In [47]:
feature_matrix.shape

(3470, 174)

In [48]:
cols = list(feature_matrix)
nunique = feature_matrix.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
feature_matrix.drop(cols_to_drop, axis=1,inplace=True)

In [49]:
feature_matrix.shape

(3470, 174)

#### drop cols that are duplicated 

In [50]:
from utils import duplicate_columns

In [51]:
dups = duplicate_columns(feature_matrix)
dups

['WEEKEND(first_EDatt_time)',
 'LAST(EDatt.WEEKEND(arrival))',
 'MONTH(first_EDatt_time)',
 'YEAR(first_EDatt_time)',
 'WEEKDAY(first_EDatt_time)',
 'LAST(EDatt.MONTH(arrival))',
 'LAST(EDatt.YEAR(arrival))',
 'LAST(EDatt.WEEKDAY(arrival))',
 'MEAN(hourly.IPadm_daycase)',
 'SUM(hourly.IPadm_daycase)',
 'MEAN(EDatt.patients.MAX(EDatt.age))',
 'SUM(EDatt.patients.MAX(EDatt.age))',
 'MAX(EDatt.patients.MAX(EDatt.age))',
 'LAST(EDatt.patients.MAX(EDatt.age))']

In [52]:
feature_matrix.drop(dups,axis=1,inplace=True)

In [53]:
feature_matrix.shape

(3470, 160)

### save out

In [54]:
pd.to_pickle(feature_matrix,'./activity.pkl')

In [55]:
ft.list_primitives().head(40)

,name,type,description
0,all,aggregation,Test if all values are 'True'.
1,num_unique,aggregation,Returns the number of unique categorical varia...
2,last,aggregation,Returns the last value.
3,percent_true,aggregation,Finds the percent of 'True' values in a boolea...
4,median,aggregation,Finds the median value of any feature with wel...
5,mean,aggregation,Computes the average value of a numeric feature.
6,mode,aggregation,Finds the most common element in a categorical...
7,std,aggregation,Finds the standard deviation of a numeric feat...
8,trend,aggregation,Calculates the slope of the linear trend of va...
9,count,aggregation,Counts the number of non null values.


In [56]:
ft.list_primitives().tail(40).loc[22].description

'Transform Datetime feature into the boolean of Weekend.'

# Dev

In [57]:
break

SyntaxError: 'break' outside loop (<ipython-input-57-6aaf1f276005>, line 1)

In [ ]:
ED.columns

#### create and link day table - by adding a new df

from core import create_timeseries_from_events

ED.columns

start = ED.tail(i).arrival.min().round('D')
end = ED.tail(i).arrival.max().round('D')

occED = create_timeseries_from_events(ED.tail(i),'arrival','time_leaving_dept_inc_cdu',col_to_split='admission_flag',start=start,end=end,freq='H')

occED['EDocc_MEAN'] = occED.sum(axis=1) # make agg col

occED['day'] = occED.index.round('D')

occEDday = occED.groupby(['day']).mean()

occEDday.index

day = occEDday['EDocc_MEAN'].reset_index()

day.head(2)

#### add to es

es = es.entity_from_dataframe(entity_id="days",dataframe=day,
                              index="day",
                              time_index="day")

new_relationship = ft.Relationship(es["days"]["day"],
                                   es["ED_atten"]["arr_date"])

es = es.add_relationship(new_relationship)

#### Look at tables

In [ ]:
es['days'].df.head()

es['days'].df['day'].head()

es['days'].df['day'].dtype

es['ED_atten'].index

es['ED_atten'].df.arrival.dtype

es['patient'].df.columns #.dtype

es['patient'].df['first_ED_atten_time'].dtype

In [ ]:
es['ED_atten']

In [ ]:
es['patient']

In [ ]:
es['days']

es['days']

In [ ]:
es

# DFS

### create range of datetimes for arrival in df

In [ ]:
feature_matrix, feature_defs = ft.dfs(entityset=es,target_entity="days")

In [ ]:
feature_matrix

In [ ]:
window_fm, window_features = ft.dfs(entityset=es,target_entity='days',cutoff_time=ct,
       cutoff_time_in_index=True,
      training_window = '24 hours')

In [ ]:
pd.DatetimeIndex(start=start,end=end,freq='D')

In [ ]:
ft.list_primitives().head(40)